# HW 1
The homework assignments is on calibrating the Inertial sensor in our cell phones via the 6-position method. The first part will be to use recorded data to calibrate the accelerometer, the second part will be to calibrate the gyroscopes and the third part will be to analyze recorded data of "shaking the smartphone". Finally, we will compare different phones with one another and with our expectations from their specifications. 


In [1]:
import sys
import os
import numpy as np
import pandas as pd

sys.path.append('../src')
from ins import Accelerometer
from ins import Gyroscope
sys.path.append('../notebooks')
np.set_printoptions(precision=4, suppress=True)

# Load the data in old format
path_accel_300sec_face_up = os.path.join("..", "data", "Elad", "old_format", "Acceleration_300sec_FaceUp")
path_accel_300sec_face_down = os.path.join("..", "data", "Elad", "old_format", "Acceleration_300sec_FaceDown")
path_accel_300sec_side_positive = os.path.join("..", "data", "Elad", "old_format", "Acceleration_300sec_SidePositive")
path_accel_300sec_side_negative = os.path.join("..", "data", "Elad", "old_format", "Acceleration_300sec_SideNegative")
path_accel_300sec_vertical_front = os.path.join("..", "data", "Elad", "old_format", "Acceleration_300sec_VerticalFront")
path_accel_300sec_vertical_back = os.path.join("..", "data", "Elad", "old_format", "Acceleration_300sec_VerticalBack")

elad_acceleration_with_g_300sec_up = pd.read_csv(os.path.join(path_accel_300sec_face_up, "Raw Data.csv")) # This had a delimiter of ","
elad_acceleration_with_g_300sec_down = pd.read_csv(os.path.join(path_accel_300sec_face_down, "Raw Data.csv"), sep="\t", engine="python")
elad_acceleration_with_g_300sec_left = pd.read_csv(os.path.join(path_accel_300sec_side_positive, "Raw Data.csv"), sep="\t", engine="python")
elad_acceleration_with_g_300sec_right = pd.read_csv(os.path.join(path_accel_300sec_side_negative, "Raw Data.csv"), sep="\t", engine="python")
elad_acceleration_with_g_300sec_front = pd.read_csv(os.path.join(path_accel_300sec_vertical_front, "Raw Data.csv"), sep="\t", engine="python")
elad_acceleration_with_g_300sec_back = pd.read_csv(os.path.join(path_accel_300sec_vertical_back, "Raw Data.csv"), sep="\t", engine="python")

# Load the data in new format
def load_data(person):
    for folder in os.listdir(os.path.join("..", "data", person)):
        df_name = f"{person.lower()}_{folder.lower().replace(' ', '_')}"
        if "Raw Data.csv" in os.listdir(os.path.join("..", "data", person, folder)):
            globals()[df_name] = pd.read_csv(os.path.join("..", "data", person, folder, "Raw Data.csv"))
            print(f"Loaded {df_name} in {folder}")
        elif "Accelerometer.csv" in os.listdir(os.path.join("..", "data", person, folder)):
            # This case means there is also gyroscope data
            globals()[df_name+"_accelerometer"] = pd.read_csv(os.path.join("..", "data", person, folder, "Accelerometer.csv"))
            print(f"Loaded {df_name}_accelerometer in {folder}")
            globals()[df_name+"_gyroscope"] = pd.read_csv(os.path.join("..", "data", person, folder, "Gyroscope.csv"))
            print(f"Loaded {df_name}_gyroscope in {folder}")
        else:
            print(f"Could not find data for {person} in {folder}")


load_data("Ben")
load_data("Elad")

Loaded ben_acceleration_with_g_right_5 in Acceleration with g RIGHT 5
Loaded ben_accelerometer_and_gyroscope_60_accelerometer in Accelerometer and Gyroscope 60
Loaded ben_accelerometer_and_gyroscope_60_gyroscope in Accelerometer and Gyroscope 60
Loaded ben_acceleration_with_g_stand_5 in Acceleration with g STAND 5
Loaded ben_accelerometer_and_gyroscope_aftershake_300_accelerometer in Accelerometer and Gyroscope afterShake_300
Loaded ben_accelerometer_and_gyroscope_aftershake_300_gyroscope in Accelerometer and Gyroscope afterShake_300
Loaded ben_acceleration_with_g_60 in Acceleration with g 60
Loaded ben_acceleration_with_g_300_flip in Acceleration with g 300_flip
Loaded ben_acceleration_with_g_stand_60 in Acceleration with g STAND 60
Loaded ben_acceleration_with_g_left_300 in Acceleration with g LEFT 300
Loaded ben_acceleration_with_g_right_300 in Acceleration with g RIGHT 300
Loaded ben_accelerometer_and_gyroscope_aftershake_60_accelerometer in Accelerometer and Gyroscope afterShake_6

In [3]:
elad_acceleration_with_g_300sec_left.head()

,Time (s),Acceleration x (m/s^2),Acceleration y (m/s^2),Acceleration z (m/s^2),Absolute acceleration (m/s^2)
0,0.020750,9.677472,-0.107677,0.024227,9.678101
1,0.023176,9.663116,-0.064606,0.045763,9.663440
2,0.025602,9.663116,-0.071785,0.050548,9.663514
3,0.028027,9.675079,-0.069392,0.052941,9.675473
4,0.030453,9.639187,-0.066999,0.043370,9.639517


In [2]:
def remove_rows_by_time(df, start_time, end_time):
    return df[(df["Time (s)"] >= start_time) & (df["Time (s)"] <= end_time)]

def get_mean_accelerations(df):
    return df[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])


# Select the first 5 seconds of data
fz_up_5sec = remove_rows_by_time(accel_300sec_face_up, 0, 5)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fz_down_5sec = remove_rows_by_time(accel_300sec_face_down, 0, 5)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fx_up_5sec = remove_rows_by_time(accel_300sec_side_positive, 0, 5)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fx_down_5sec = remove_rows_by_time(accel_300sec_side_negative, 0, 5)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fy_up_5sec = remove_rows_by_time(accel_300sec_vertical_front, 0, 5)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fy_down_5sec = remove_rows_by_time(accel_300sec_vertical_back, 0, 5)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])

# Select the first 60 seconds of data
fz_up_60sec = remove_rows_by_time(accel_300sec_face_up, 0, 60)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fz_down_60sec = remove_rows_by_time(accel_300sec_face_down, 0, 60)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fx_up_60sec = remove_rows_by_time(accel_300sec_side_positive, 0, 60)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fx_down_60sec = remove_rows_by_time(accel_300sec_side_negative, 0, 60)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fy_up_60sec = remove_rows_by_time(accel_300sec_vertical_front, 0, 60)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fy_down_60sec = remove_rows_by_time(accel_300sec_vertical_back, 0, 60)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])

# Select the first 300 seconds of data
fz_up_300sec = remove_rows_by_time(accel_300sec_face_up, 20, 300)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fz_down_300sec = remove_rows_by_time(accel_300sec_face_down, 0, 300)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fx_up_300sec = remove_rows_by_time(accel_300sec_side_positive, 0, 300)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fx_down_300sec = remove_rows_by_time(accel_300sec_side_negative, 0, 300)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fy_up_300sec = remove_rows_by_time(accel_300sec_vertical_front, 0, 300)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])
fy_down_300sec = remove_rows_by_time(accel_300sec_vertical_back, 0, 300)[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].mean().to_numpy().reshape([3,1])


### Part 1 - Calculate bias and scale factor assuming no misalignment

In [3]:
g_true = 9.81 # m/s^2

# create sensors objects for the accelerometers for the different time periods (5, 60, 300 seconds)
Elad_acc_300 = Accelerometer(name="Elad accelerometer 300 sec")
Elad_acc_300.set_calibration_data(
    fx_up=fx_up_300sec, fx_down=fx_down_300sec, 
    fy_up=fy_up_300sec, fy_down=fy_down_300sec, 
    fz_up=fz_up_300sec, fz_down=fz_down_300sec, 
    g=g_true)

Elad_acc_60 = Accelerometer(name="Elad accelerometer 60 sec")
Elad_acc_60.set_calibration_data(
    fx_up=fx_up_60sec, fx_down=fx_down_60sec, 
    fy_up=fy_up_60sec, fy_down=fy_down_60sec, 
    fz_up=fz_up_60sec, fz_down=fz_down_60sec, 
    g=g_true)

Elad_acc_5 = Accelerometer(name="Elad accelerometer 5 sec")
Elad_acc_5.set_calibration_data(
    fx_up=fx_up_5sec, fx_down=fx_down_5sec, 
    fy_up=fy_up_5sec, fy_down=fy_down_5sec, 
    fz_up=fz_up_5sec, fz_down=fz_down_5sec, 
    g=g_true)



In [4]:
bias = Elad_acc_300.compute_bias()
bias = Elad_acc_60.compute_bias()
bias = Elad_acc_5.compute_bias()
scale_factor = Elad_acc_300.compute_scale_factor()
scale_factor = Elad_acc_60.compute_scale_factor()
scale_factor = Elad_acc_5.compute_scale_factor()

In [ ]:
# Compute bias and scale factor for each accelerometer
bias_300 = Elad_acc_300.compute_bias()
scale_factor_300 = Elad_acc_300.compute_scale_factor()

bias_60 = Elad_acc_60.compute_bias()
scale_factor_60 = Elad_acc_60.compute_scale_factor()

bias_5 = Elad_acc_5.compute_bias()
scale_factor_5 = Elad_acc_5.compute_scale_factor()

# Print stats for each accelerometer
print(f"Stats for {Elad_acc_300.get_sensor_name()} sensors:")
print(f"bias x: {bias_300[0][0]}\nbias y: {bias_300[1][0]}\nbias z: {bias_300[2][0]}")
print(f"scale factor x: {scale_factor_300[0,0]}\nscale factor y: {scale_factor_300[1,1]}\nscale factor z: {scale_factor_300[2,2]}")

print(f"Stats for {Elad_acc_60.get_sensor_name()} sensors:")
print(f"bias x: {bias_60[0][0]}\nbias y: {bias_60[1][0]}\nbias z: {bias_60[2][0]}")
print(f"scale factor x: {scale_factor_60[0,0]}\nscale factor y: {scale_factor_60[1,1]}\nscale factor z: {scale_factor_60[2,2]}")

print(f"Stats for {Elad_acc_5.get_sensor_name()} sensors:")
print(f"bias x: {bias_5[0][0]}\nbias y: {bias_5[1][0]}\nbias z: {bias_5[2][0]}")
print(f"scale factor x: {scale_factor_5[0,0]}\nscale factor y: {scale_factor_5[1,1]}\nscale factor z: {scale_factor_5[2,2]}")


### Part 2 - Calculate Scale Factor, Bias and Misalignment using LSM

In [6]:
# Calculate the errors matrix M = [SF + M | b]
M300 = Elad_acc_300.compute_M_errors_matrix()
M60 = Elad_acc_60.compute_M_errors_matrix()
M10 = Elad_acc_5.compute_M_errors_matrix()

In [ ]:
print(f"Stats for {Elad_acc_300.get_sensor_name()} sensors:")
print(f"M Matrix:\n{M300}")

print(f"Stats for {Elad_acc_60.get_sensor_name()} sensors:")
print(f"M Matrix:\n{M60}")

print(f"Stats for {Elad_acc_5.get_sensor_name()} sensors:")
print(f"M Matrix:\n{M10}")

In [ ]:
xdata = accel_300sec_face_up["Time (s)"].to_numpy()
ydata = accel_300sec_face_up[["Acceleration x (m/s^2)", "Acceleration y (m/s^2)", "Acceleration z (m/s^2)"]].to_numpy()
Elad_acc_300.plot_time_series(xdata, ydata, "Accelerations (m/s^2)", "Time (s)")


In [ ]:
accel_300sec_face_up.head()